## Imports

Outside of scanpy, which is a python library for analyzing scRNAseq these rest of these should be standard computational python libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt

import statsmodels.api as sm

If you need to install scanpy here is a cell for that

In [ ]:
#!pip install scanpy

In [ ]:
import scanpy as sc

## Mann Whitney Test

To illustrate the speedup you can get from vectorization I we will talk about the MannWhitney test because the default function for doing this in python (`stats.mannwhitneyu`) is not vectorized. However, as long as you know the math behind the function it is pretty straightforward to code it up and get a big speedup.

The test is often used to find marker genes in scRNAseq data. This analysis is generally called Differential Expression (DE). By testing the differnetial expression for every gene between cell-types you can identify genes that identify the cell type. There are many ways to compute this. Parametrically this is usually done using a glm, but you can do it non-parametrically with a MannWhitney test.

Today we are using some sample data from scanpy that is scRNAseq of mouse bone marrow. 



In [ ]:
paul = sc.datasets.paul15()

This data is stored as an AnnData object

<img src="https://falexwolf.de/img/scanpy/anndata.svg" />

For this analysis you just need the expression data and the cluster labels. The expression data is stored in cells x genes under `.X` and the clusters are stored in `.obs` as `paul15_clusters`

In [ ]:
clusters = paul.obs.paul15_clusters
ax = sns.countplot(clusters)
ax.set_xticklabels(ax.get_xticklabels(),ha='right',rotation=45)
sns.despine()

Differential expression can be done by either comparing each cluster to all other clusters, known as 1vAll or done using a 1v1 approach, where you compute all pairwise comparisions between the clusters. 

For this exercise we can just select 2 clusters and compare them. I selected two Monocyte clusters, 14Mo and 15Mo but you can use any pair of clusters you want

In [ ]:
##Getting the clusters you need
selected_clusters = clusters.isin(['14Mo', '15Mo'])

cluster_labels = clusters[selected_clusters]
cluster14 = cluster_labels.index[cluster_labels == '14Mo']
cluster15 = cluster_labels.index[cluster_labels == '15Mo']

In [ ]:
##Subsetting the expression data (stored)
expression = paul.to_df()[selected_clusters]
expression.shape

To illustrate the speedup we get from vectorization we need to first compute differntial expression p values using `stats.mannwhitneyu` 

> Note: It is common for gene expression to be 0 for all cells for a given gene, which will throw an error in the `stats.mannwhitneyu`

In [ ]:
%%timeit
## TODO: Compute p values for DE using stats.mannwhitneyu for all genes

### Calculating Mann Whitney U test statistic:

There are a few equations you need for computing significance values for this. 

1. Compute U

$\large{U = R - ((n_1 * (n_1 + 1)) /2)}$

$n_1$ = sample size of group 1

$n_2$ = sample size of group 2

R is sum of the ranks of (order) of the values of group 1

2. Compute Z

$\large{Z = \frac{|U - \frac{n_1 * n_2}{2}|}{\sqrt{\frac{n_1 * n_2 * (n_1 + n_2 + 1)}{12}}}}$

Note the above equation doesn't use tie correction, which is lightly different than the `stats.mannwhitenyu` function. Ranks are computed by giving ties the average rank of all the values, which slightly messes up the standard deviation of the uniform distribution. This correction isn't totally necessary, and excluding it just makes your p values more conservative 

$\large{Z = \frac{U - \textit{m}_U}{\sigma_U}}$

$\large{\textit{m}_U = \frac{n_1n_2}{2}}$

$\large{\sigma_{corr} = \sqrt{\frac{n_1n_2}{12}((n+1) - \sum\limits_{i=1}^k{\frac{t_i^3 - t_i}{n(n-1)}})}}$

$n = n_1 + n_2$

$t_i$ is the number of subjects sharing rank i, and k is the number of (distinct) ranks

3. Use normal distribution to compute p values

In [ ]:
%%timeit
##TODO Compute p values for all genes using vectorization

These are uncorrected p values, which aren't terribly useful, so to correct them you can use the Benjamini Hotchberg procedure using `sm.stats.multipletests ` (in statsmodels)


In [ ]:
## TODO Compute adjusted p values

When doing differential expression it often isn't enough to just look at the p values to discern markers. So we often use a second metric, log2FoldChange. 

You can compute log2FC by taking the log of the ratios of average expression of each group

In [ ]:
## Compute log2FC for all genes

If you want to visualize these results you can create what is known as a volcano plot, where you create a scatterplot with log2FoldChange on the x axis and -log10padj.

In [ ]:
## Create a volcano plot